# Physical Models: Diffusion and Percolation
This is our week 7 examples notebook and will be available on Github from the powderflask/cap-comp215 repository.
As usual, the first code block just imports the modules we will use.

In [64]:
import math
from itertools import tee
import matplotlib.pyplot as plt
from matplotlib import animation, rc, lines
import numpy as np
from scipy.signal import correlate2d

### Helpers
The Point2D class we examined in class in week 6 and a lattice random walk function for Point2D objects...


In [65]:
class Point2D:
    """ A point on the 2D Cartesian plane """
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return '({x}, {y})'.format(x=round(self.x,2), y=round(self.y,2))

    def __add__(self, other):
        return Point2D(self.x+other.x, self.y+other.y)

    @property
    def r(self):
        """ length, r, from origin to this point """
        return math.sqrt(self.x**2 + self.y**2)

# N, S, E, W
CARDINAL_NEIGHBOURS = (Point2D(0,1), Point2D(0,-1), Point2D(1,0), Point2D(-1,0))

def random_walk(p):
    """ "walk" point p 1-step, to a neighbouring cardinal locaction (lattice random walk) """
    return p + np.random.choice(CARDINAL_NEIGHBOURS)

## Diffusion by "Brownian Motion"
As the length of a random walk tends to zero, it models a "Wiener process", which is a model for Brownian Motion. (https://en.wikipedia.org/wiki/Random_walk)

In the animation below, we "walk' a large set of "particles" that all start at the centre of the plane

In [67]:
class ParticleAnimation:
  """
    Animates a set of particles
  """
  def __init__(self, step, n_particles=1, show_lines=False, frames=50, figsize=(8, 8)):
    """
    :param step: a function that takes a single Point2D and returns one Point2D (presumably in next state)
    :param n_particles: number of particles to include in simulation
    :param frames: number of animation frames to generate
    """
    self.step = step
    self.ax_lim = (-frames//4, frames//4)   # particles will diffuse over wider area as animation goes on longer
    # start all particles in the centre
    self.particles = [Point2D(0, 0)for _ in range(n_particles)]
    self.radius = 0

    self.show_lines = show_lines
    self.lines = [(p, ) for p in self.particles]

    fig, self.ax = plt.subplots(figsize=figsize)
    self.animation = animation.FuncAnimation(fig, self.animate, frames=frames)

  def draw(self):
    """ Draw the particles in their current state """
    x, y = zip(*((p.x, p.y) for p in self.particles))
    self.ax.scatter(list(x), list(y), c=[i for i in range(len(x))], cmap=plt.get_cmap('tab20'))
    # fix the axes limits so they don't autoscale as particles diffuse
    self.ax.set_ylim(*self.ax_lim)
    self.ax.set_xlim(*self.ax_lim)
    # TODO: draw circle with self.radius
    if self.show_lines:
        for line in self.lines:
            x, y = zip(*((p.x, p.y) for p in line))
            self.ax.plot(x, y)

  def animate(self, step):
    """ Step the model forward and draw the ploy """
    if step > 0:
        self.particles = [self.step(p) for p in self.particles]
        self.radius = max(p.r for p in self.particles)
        self.lines = [l+(p,) for l,p in zip(self.lines, self.particles)]
    self.ax.clear()
    self.ax.set_title("Step {step}  Raidus {r}".format(step=step, r=round(self.radius)))
    self.draw()

  def show(self):
    """ return the matplotlib animation, ready for display """
    plt.close()
    rc('animation', html='jshtml')
    return self.animation


# Animate a collection of random walking particles...
n_particles = 100
frames = 50
show_lines = False

anim = ParticleAnimation(random_walk, n_particles, show_lines=show_lines, frames=frames)
anim.show()

## Percolation

Numpy arrays have some really handy capabilities.
For example, you can get an array of booleans that match some criteria, like this...

In [77]:
a = np.array((0, 1, 6, 2, 9, 4, 0, 2, 4, 7,))
a > 5

array([False, False,  True, False,  True, False, False, False, False,
        True])

and you can use an array of booleans like this to filter the array, giving you a dis-continuous "slice" with only elements that meet the criteria...

In [78]:
a[a > 5]

array([6, 9, 7])

The operators ``&`` (and) and ``|`` (or) can be used build aribitrarily complex filters.  Be sure to bracket sub-expressions since the ``&`` and ``or`` operators have high precedence.

In [79]:
b = a*2
a[(a>5) & (b<15)]

array([6, 7])

In [81]:
a[(a<3) | (b>15)]

array([0, 1, 2, 9, 0, 2])

The resulting array is actually a slice of the original, so any mutation applies to the original array!!

In [82]:
a[a>5] = 5
a

array([0, 1, 5, 2, 5, 4, 0, 2, 4, 5])